# Insurance Premium Cost Prediction

Predict medical insurance charges from age, BMI, smoking status, and region.

**Dataset:** [https://www.kaggle.com/datasets/mirichoi0218/insurance](https://www.kaggle.com/datasets/mirichoi0218/insurance)  
**Target:** `charges`  
**Type:** Regression

> **TODO:** Download the dataset, place it in `../../data/raw/`, then update `DATA_PATH` and `TARGET` below.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
sns.set_theme(style='whitegrid')

## 1. Load Data

In [ ]:
# TODO: update path after downloading from https://www.kaggle.com/datasets/mirichoi0218/insurance
DATA_PATH = "../../data/raw/insurance.csv"
TARGET = "charges"  # TODO: verify column name

df = pd.read_csv(DATA_PATH)
print(f'Shape: {df.shape}')
df.head()

## 2. Exploratory Data Analysis

In [ ]:
print(df.info())
print('\nNull counts:')
print(df.isnull().sum().sort_values(ascending=False).head(15))
df.describe().T

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
df[TARGET].hist(bins=40, ax=axes[0])
axes[0].set_title(f'Distribution: {TARGET}')
np.log1p(df[TARGET]).hist(bins=40, ax=axes[1])
axes[1].set_title(f'Log Distribution: {TARGET}')
plt.tight_layout(); plt.show()
print(df[TARGET].describe())

In [ ]:
# Correlation with target
num_df = df.select_dtypes(include='number')
corr = num_df.corr()[TARGET].drop(TARGET).sort_values()
corr.plot(kind='barh', figsize=(8, max(4, len(corr) * 0.3)))
plt.title(f'Feature Correlation with {TARGET}')
plt.tight_layout(); plt.show()

## 3. Feature Engineering

In [ ]:
X = df.drop(columns=[TARGET])
y = df[TARGET]

# Optional: log-transform skewed target
# y = np.log1p(y)

numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])
preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols),
])

## 4. Train / Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f'Train: {X_train.shape}, Test: {X_test.shape}')

## 5. Model Training

In [ ]:
def eval_reg(name, pipe, X_tr, X_te, y_tr, y_te):
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)
    mae = mean_absolute_error(y_te, preds)
    rmse = np.sqrt(mean_squared_error(y_te, preds))
    r2 = r2_score(y_te, preds)
    print(f'{name}: MAE={mae:.3f}  RMSE={rmse:.3f}  R²={r2:.4f}')
    return pipe, preds, {'mae': mae, 'rmse': rmse, 'r2': r2}

models = {
    'Linear Regression': Pipeline([('pre', preprocessor), ('reg', LinearRegression())]),
    'Ridge': Pipeline([('pre', preprocessor), ('reg', Ridge(alpha=1.0))]),
    'Random Forest': Pipeline([('pre', preprocessor),
                              ('reg', RandomForestRegressor(n_estimators=100, random_state=42))]),
    'Gradient Boosting': Pipeline([('pre', preprocessor),
                                  ('reg', GradientBoostingRegressor(n_estimators=100, random_state=42))]),
}

results = {}
for name, pipe in models.items():
    fitted, preds, metrics = eval_reg(name, pipe, X_train, X_test, y_train, y_test)
    results[name] = {'pipe': fitted, 'preds': preds, 'metrics': metrics}

## 6. Evaluation

In [ ]:
best_name = min(results, key=lambda k: results[k]['metrics']['rmse'])
best_preds = results[best_name]['preds']
print(f'Best model: {best_name}')

# Actual vs Predicted
fig, axes = plt.subplots(1, 2, figsize=(13, 5))
axes[0].scatter(y_test, best_preds, alpha=0.4, s=15)
lims = [min(y_test.min(), best_preds.min()), max(y_test.max(), best_preds.max())]
axes[0].plot(lims, lims, 'r--')
axes[0].set_xlabel('Actual'); axes[0].set_ylabel('Predicted')
axes[0].set_title(f'Actual vs Predicted — {best_name}')

# Residuals
residuals = y_test - best_preds
axes[1].scatter(best_preds, residuals, alpha=0.4, s=15)
axes[1].axhline(0, color='r', linestyle='--')
axes[1].set_xlabel('Predicted'); axes[1].set_ylabel('Residual')
axes[1].set_title('Residual Plot')
plt.tight_layout(); plt.show()

In [ ]:
# Feature importances
rf_pipe = results['Random Forest']['pipe']
feat_names = rf_pipe.named_steps['pre'].get_feature_names_out()
importances = pd.Series(
    rf_pipe.named_steps['reg'].feature_importances_, index=feat_names
)
importances.nlargest(15).sort_values().plot(kind='barh', figsize=(8, 5))
plt.title('Top 15 Feature Importances (Random Forest)')
plt.tight_layout(); plt.show()

## 7. Conclusion

| Model | MAE | RMSE | R² |
|---|---|---|---|
| *(fill after running)* | | | |

**Observations:**
- 

**Next steps:**
- Log-transform skewed target if not done
- Hyperparameter tuning
- Try XGBoost / LightGBM